# Materials Graph Networks (MEGNET)
### Sheth Riya Nimish
### A0176880R
e0235287@u.nus.edu

### Source

#### Graph Networks as a Universal Machine Learning Framework for Molecules and Crystals
 Chi Chen, Weike Ye, Yunxing Zuo, Chen Zheng, and Shyue Ping Ong

#### The code here is to execute the materials graph network. The code here has been written by me from scratch while the model used is a published model. 

The model was published in the following github repository: https://github.com/materialsvirtuallab/megnet

#### Table of Contents

* [Importing the Required Libraries](#import)
* [Creating a MEGNET Model](#create)
* [Reading the Required Data From a File](#read)
* [Extracting the Structures from the CIF files](#extract)
* [Training the Model](#train)
* [Calculating the Accuracy](#acc)

The MatErials Graph Network (MEGNet) is an implementation of DeepMind's graph networks[1] for universal machine learning in materials science. 

### Importing the Required Libraries<a class="anchor" id="import"></a>

In [1]:
from pymatgen import Structure, Lattice
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt

### Creating a MEGNET Model <a class="anchor" id="create"></a>

In [3]:
# a list of the parameters which can be optimised and fine tuned
nfeat_bond = 10 
r_cutoff = 5
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)
model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)

### Reading the Required Data from a File<a class="anchor" id="read"></a>

In [4]:
filename= input("Enter File Name:")
tp= pd.read_csv(filename)
filenames= tp['Material'].values.tolist()
propertylist= tp['Property'].values.tolist()
actual= tp['Actual'].values.tolist()

Enter File Name:datamegnet.csv


### Extracting the Structures from the CIF files <a class="anchor" id="extract"></a>

In [6]:
structure=[]
for i in range (1, len(filenames)):
    temp= "cifmegnet/" +filenames[i]+ ".cif"
    structure.append(Structure.from_file(temp))

/Users/riyasheth/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1123: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


### Training the model <a class="anchor" id="train"></a>

In [7]:
model.train(structure, propertylist, epochs=100)

Epoch 1/100


/Users/riyasheth/.local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/riyasheth/.local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of

23/23 [==============================] - 43s 986ms/step - loss: 0.4063
Epoch 2/100
23/23 [==============================] - 23s 1s/step - loss: 0.2585
Epoch 3/100
23/23 [==============================] - 22s 935ms/step - loss: 0.2515
Epoch 4/100
23/23 [==============================] - 20s 871ms/step - loss: 0.2494
Epoch 5/100
23/23 [==============================] - 20s 873ms/step - loss: 0.2510
Epoch 6/100
23/23 [==============================] - 20s 871ms/step - loss: 0.2489
Epoch 7/100
23/23 [==============================] - 22s 920ms/step - loss: 0.2487
Epoch 8/100
23/23 [==============================] - 20s 870ms/step - loss: 0.2462
Epoch 9/100
23/23 [==============================] - 21s 901ms/step - loss: 0.2465
Epoch 10/100
23/23 [==============================] - 23s 964ms/step - loss: 0.2464
Epoch 11/100
23/23 [==============================] - 20s 873ms/step - loss: 0.2439
Epoch 12/100
23/23 [==============================] - 20s 874ms/step - loss: 0.2430
Epoch 13/100
23/

23/23 [==============================] - 20s 857ms/step - loss: 0.2372
Epoch 100/100
23/23 [==============================] - 20s 855ms/step - loss: 0.2385


### Calculating the Accuracy <a class="anchor" id="acc"></a>

In [8]:
def calculate_accuracy():
    choice= [0, 1]
    count=0
    for i in range (0, len(structure)):
        temp= model.predict_structure(structure[i])
        t= temp[0]
        result= random.choices(choice, weights= (1-t, t), k=1) 
        #since this is a probabilistic classification, there should be a randomly weighted choice based on the
        #probability predicted by the model
        if result[0] == propertylist[i]:
            count= count+1
            
    accuracy= count/len(structure)
    print("The accuracy is", accuracy)
    return accuracy
calculate_accuracy()

The accuracy is 0.5249738037024101


0.5249738037024101